In [1]:
import pandas as pd

df_road = pd.read_csv("/Users/2020/road-to-dong/pos00005m.csv")
df_dong = pd.read_csv("/Users/2020/road-to-dong/pos00006m.csv")

df_road = df_road[['rd_nm', 'bld_hmno', 'bld_hsno', 'lgdng_cd']]
df_dong = df_dong[df_dong['ctpv_nm']=='서울특별시']

def bld_split(bd_nm):
    if '-' in bd_nm:
        bld_hmno, bld_hsno = bd_nm.split('-')
    else:
        bld_hmno = bd_nm
        bld_hsno = '0'
    return bld_hmno, bld_hsno

# raod : str type
def get_rd_bd(road_addr):
    temp_list = road_addr.split()
    rd_nm = temp_list[2]
    bld_hmno, bld_hsno = bld_split(temp_list[3])
    if bld_hmno.isdecimal() & bld_hsno.isdecimal():
        return rd_nm, int(bld_hmno), int(bld_hsno)
    else:
        return rd_nm, 0, 0  

# road_bd 예시 : '테헤란로6길', 30, 0 ( '길' '건물본번' '건물부번' type:str, int, int)
def rd_bd_to_dong(rd_nm, bld_hmno, bld_hsno):
    if bld_hmno==0:
        return '검색불가' # 오류 : 잘못 기록된 도로명
    df_temp = df_road[df_road['rd_nm']==rd_nm]
    df_temp = df_temp[df_temp['bld_hmno']==bld_hmno]
    df_temp = df_temp[df_temp['bld_hsno']==bld_hsno]
    if len(df_temp) == 0:
        return '검색불가' # 검색 테이블에 존재x
    for i in range(len(df_temp['lgdng_cd'])):
        lgdng_cd = df_temp['lgdng_cd'].iat[i]
        df_temp2 = df_dong[df_dong['lgdng_cd']==lgdng_cd]
        if len(df_temp2) != 0:
            return df_temp2.iloc[0]['lgdng_nm']
    return '검색불가'# 서울 외 지역

# 도로명주소(상세)-> 법정동 이름
def road_address_to_dong(road_addr):
    return rd_bd_to_dong(*get_rd_bd(road_addr))

#road_address to dong (series)
def rtds(pd_series):
    for i in range(len(pd_series)):
        pd_series[i] = road_address_to_dong(pd_series[i])
    return pd_series

### Test

In [2]:
road_address_to_dong('서울특별시 구로구 구일로 154-17 101동 101호')

'구로동'

In [3]:
pd.set_option('display.max_row', 1000)

In [4]:
test_series = pd.read_csv("/Users/2020/road-to-dong/서울특별시 어린이집 정보(표준 데이터).csv", encoding='cp949')
test_series = test_series['상세주소'].copy()
test_series = test_series[:500]
test_series

0                                   서울특별시 서초구 사평대로 310-4
1                                      서울특별시 서초구 효령로 391
2                            서울특별시 서초구 방배로 21 방배그랑자이어린이집
3                                  서울특별시 송파구 마천로28길 3 1층
4                                    서울특별시 구로구 부광로 88 1층
5                       서울특별시 광진구 능동로 69 어린이집(자양동자양호반써밋)
6           서울특별시 마포구 마포대로11길 50 409동 1층(공덕동 공덕래미안4차아파트)
7                                    서울특별시 송파구 오금로 62 1층
8                                    서울특별시 구로구 고척로33길 43
9                   서울특별시 강남구 테헤란로98길 12 3층(대치동현대백화점 본사)
10                                   서울특별시 강남구 논현로99길 21
11                           서울특별시 금천구 벚꽃로 100 105동 102호
12                            서울특별시 중랑구 용마산로136길 160 관리동
13                          서울특별시 서초구 사평대로 240 502동 107호
14                  서울특별시 동작구 동작대로39길 39 이수푸르지오더프레티움 관리동
15                                   서울특별시 서초구 서초중앙로 206
16                                    서울특별시 송파구 위례송파로 89
17                서울특별시 금천구 벚꽃로

In [6]:
# 분당 200개 속도 (코랩 무료버전 정도 성능)
test_series = rtds(test_series)

In [8]:
test_series

0        검색불가
1        검색불가
2        검색불가
3         오금동
4        검색불가
5        검색불가
6         공덕동
7         신천동
8         고척동
9        검색불가
10        역삼동
11       검색불가
12       검색불가
13        반포동
14       검색불가
15       검색불가
16       검색불가
17       검색불가
18       검색불가
19       검색불가
20        연남동
21       검색불가
22        자양동
23        중곡동
24        갈현동
25        화곡동
26        고척동
27        등촌동
28       검색불가
29       검색불가
30        장지동
31        홍은동
32       검색불가
33      신문로2가
34       검색불가
35        대치동
36        역삼동
37       검색불가
38      한강로1가
39       검색불가
40       검색불가
41        상도동
42        면목동
43       검색불가
44        개봉동
45         본동
46        방배동
47        쌍문동
48        개봉동
49        방학동
50       답십리동
51       검색불가
52        화곡동
53        시흥동
54        도봉동
55         창동
56       검색불가
57        서초동
58       검색불가
59       검색불가
60         목동
61         창동
62       검색불가
63        반포동
64       검색불가
65        일원동
66       여의도동
67        풍납동
68        역촌동
69        삼전동
70       검색불가
71    

In [7]:
# 총 500개
test_series.value_counts()

검색불가      83
화곡동       16
가락동       14
창동        10
신정동       10
상계동        9
미아동        8
진관동        8
봉천동        8
항동         8
남가좌동       8
면목동        7
방학동        7
역삼동        6
도봉동        6
고덕동        6
고척동        6
장지동        6
등촌동        6
개봉동        6
독산동        6
월계동        6
방배동        6
갈현동        5
신내동        5
상일동        5
오금동        5
답십리동       5
상도동        5
자양동        5
시흥동        5
장위동        5
거여동        5
사당동        5
상암동        5
흑석동        4
마곡동        4
신림동        4
반포동        4
신도림동       4
공릉동        4
구로동        4
구의동        4
대치동        4
길음동        4
응암동        3
상봉동        3
장안동        3
여의도동       3
신길동        3
풍납동        3
목동         3
신사동        3
홍은동        3
서초동        3
불광동        3
묵동         3
번동         3
연희동        2
대림동        2
신당동        2
내발산동       2
우면동        2
일원동        2
방이동        2
신월동        2
가양동        2
광장동        2
역촌동        2
녹번동        2
하왕십리동      2
암사동        2
구산동        2
휘경동        2
전농동        2
천호동        2
수유동        2